In [0]:
!pip install -q pyinterval

In [0]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import queue
import pydot
import json
from interval import interval, inf, imath
from networkx.drawing.nx_pydot import graphviz_layout
from collections import deque, OrderedDict
from google.colab import files
from sympy.solvers import solve
from sympy import Symbol

In [0]:
def inv_matrix(matrix) :
  det = matrix[0][0]*matrix[1][1] - matrix[0][1]*matrix[1][0]
  print('Determinant: ', det)
  a11 = matrix[1][1]/det
  a12 = -matrix[0][1]/det
  a21 = -matrix[1][0]/det
  a22 = matrix[0][0]/det
  return np.array([[a11, a12], [a21, a22]])

def multiply_matrix(first, second):
  a11 = first[0][0]*second[0] + first[0][1]*second[0]
  a12 = first[1][0]*second[1] + first[1][1]*second[1]
  return [a11, a12]

In [0]:
def system_mura(x_min, x_max, y_min, y_max, e):
  print(f'-----Iteration for X[{x_min}; {x_max}] AND Y[{y_min}; {y_max}]')
  # print('CHECK')
  if x_max - x_min < e and y_max - y_min < e:
    print('FOUND X =', x_min, ' ', x_max)
    print('FOUND Y =', y_min, ' ', y_max)
    return
  X0 = interval([x_min, x_max])
  Y0 = interval([y_min, y_max])
        
  print('X0: ',  X0)
  midX = (x_min+x_max)/2; 
  midY = (y_min+y_max)/2; 
  print('middle X: ',  midX)
  print('middle Y: ',  midY)
  # function
  f = [2*midX**2 - midY - 1, midX + midY**2 - 11] 
  print('f: ',  f)
  if f[0] == 0.0 and f[1] == 0.0:
    system_mura(x_min, midX, y_min, y_max, e)
    system_mura(midX, x_max, midY, y_max, e)
    return
  print('function value:', f);
  # derivative
  f_derivative = [[4*X0, -1], [1, 2*Y0]]
  # f_derivative = np.array([[0, 2], [1, 7]]) 
  print('function deritive: '); 
  print(f_derivative);
  complex_right_matrix = multiply_matrix(inv_matrix(f_derivative), f);
  print('Inversed and multiplied: ', complex_right_matrix)
  UX = X0 - interval(complex_right_matrix[0][0]) - interval(complex_right_matrix[1][0])
  print('UX: ', UX)
  UY = Y0 - interval(complex_right_matrix[1][0]) - interval(complex_right_matrix[1][0])
  print('UY: ', UY)
  
  print('UX[0]: ', UX[0])
  print('X0: ', X0)
  print('UY[0]: ', UY[0])
  X1 = UX & X0
  print('X1: ', X1)
  Y1 = UY & Y0
  print('Y1: ', Y1)
  # [system_mura(x.inf, x.sup, Y1.inf[0], Y1.sup[1], e) for x in X1]
  for x1 in X1:
    for y1 in Y1: 
      system_mura(x1.inf, x1.sup, y1.inf, y1.sup, e)
      
print("Алгоритм Мура для системи рівнянь");
print("------------");
e = 10**(-6)

x_min = 0.5;
x_max = 1.5;
y_min = 1.2;
y_max = 2.2;

system_mura(x_min, x_max, y_min, y_max, e)

Алгоритм Мура для системи рівнянь
------------
-----Iteration for X[0.5; 1.5] AND Y[1.2; 2.2]
X0:  interval([0.5, 1.5])
middle X:  1.0
middle Y:  1.7000000000000002
f:  [-0.7000000000000002, -7.109999999999999]
function value: [-0.7000000000000002, -7.109999999999999]
function deritive: 
[[interval([2.0, 6.0]), -1], [1, interval([2.4, 4.4])]]
Determinant:  interval([5.8, 27.400000000000002])
Inversed and multiplied:  [array([[-0.08686131, -0.65172414]]), array([[ 0.70688397, -7.09568336]])]
UX:  interval([-0.12002265290712333, 9.247407500629249])
UY:  interval([-0.21376793355147283, 16.391366725396427])
UX[0]:  (-0.12002265290712333, 9.247407500629249)
X0:  interval([0.5, 1.5])
UY[0]:  (-0.21376793355147283, 16.391366725396427)
X1:  interval([0.5, 1.5])
Y1:  interval([1.2, 2.2])
-----Iteration for X[0.5; 1.5] AND Y[1.2; 2.2]
X0:  interval([0.5, 1.5])
middle X:  1.0
middle Y:  1.7000000000000002
f:  [-0.7000000000000002, -7.109999999999999]
function value: [-0.7000000000000002, -7.10999

RecursionError: ignored